In [35]:
#爬取网址：
#Request URL:https://www.lagou.com/jobs/positionAjax.json?needAdditionResult=false ‘显示频繁操作’
#Referer: https://www.lagou.com/zhaopin/Python/?labelWords=label
#Request Method: POST
#User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36

#思路整理：发送get请求，获取session-->更新session，发送post请求-->获取数据，储存-->数据多维度可视化



In [36]:
#代码：
#!/usr/bin/env python3
#encoding:utf-8


import json
import requests
import xlwt
import time

In [37]:
#获取储存职位信息的json对象，遍历获得公司名、福利待遇、工作地点、学历要求、工作类型、发布时间、职位名称、薪资、工作年限
def get_json(url,datas):
    my_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
        'Referer': 'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98?city=%E5%85%A8%E5%9B%BD&cl=false&fromSearch=true&labelWords=&suginput=',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }
    time.sleep(15)
    #获取session
    ses = requests.Session()
    #更新
    ses.headers.update(my_headers)
    ses.get('https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput=')
    content = ses.post(url = url,data = datas)
    result = content.json()
    info = result['content']['positionResult']['result']
    info_list = []
    #-----------------------------------------------------------------------------------------
    #更新点：减少重复度，将分类添加至列表，遍历列表即可
    #-----------------------------------------------------------------------------------------
    classification_list = ['positionId','city','companyFullName','companyLabelList','district','education','firstType','formatCreateTime','positionName','salary','workYear']
    for job in info:
        information = []
        for classify in classification_list:
            information.append(job[classify])
        info_list.append(information)
        
    return info_list
        

In [38]:
#编写主函数
def main():
    page = int(input('Please input the total pages number you want to acquire'))
    info_result = []
    title = ['岗位id','城市','公司全名','福利待遇','工作地点','学历要求','工作类型','发布时间','职位名称','薪资','工作年限']
    info_result.append(title)
    for x in range(1,page+1):
        # https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false
        url = 'https://www.lagou.com/jobs/positionAjax.json?needAdditionResult=false'
        #更改点：'pn':x,   将x，更改为1，否则报错！！！！
        datas = {
            'first':'false',
            'pn':x,
            'kd':'数据挖掘',
        }
        try:
            info = get_json(url,datas)
            info_result = info_result + info
            print('第%s页正常采集'%x)
        except Exception as msg:
            print('第%s页出现问题'%x)
            
        #创建workbook，即Excel
        workbook = xlwt.Workbook(encoding = 'utf-8')
        #创建表，第二参数用于确认同一个cell单元是否可以重设值
        worksheet = workbook.add_sheet('lagouzp',cell_overwrite_ok=True)
        for i,row in enumerate(info_result):
            for j,col in enumerate(row):
                worksheet.write(i,j,col)
        workbook.save('拉钩_数据挖掘.xls')
    print('code is over!!!')
        

In [39]:
#执行函数
if __name__ == '__main__':
    main()